In [1]:
#Run This block of code only once as its downloading file.
#If this doesn't work on your system. please go to https://files.grouplens.org/datasets/movielens/ml-20m.zip and download 
#the zip file and place this python file in that folder and run it
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import pathlib
path = pathlib.Path().parent.absolute()
path = str(path)
zipurl = 'https://files.grouplens.org/datasets/movielens/ml-20m.zip'
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall(path)

In [2]:
import os
import operator
import time
import pandas as pd
import numpy as np
from numpy import genfromtxt
import codecs
from numpy import linalg as LA
import pandas as pd

moviesLimt = int(input("Enter movie limit for getting the subset of the dataset \n (To get faster output please give a number between 100 and 500) ")) #No:of movie ID's
RatingsLimt = int(input("Enter ratings limit for getting the subset of the dataset \n (To get faster output please give a number between 100 and 500) ")) #No: of userId's 

Enter movie limit for getting the subset of the dataset 
 (To get faster output please give a number between 100 and 500) 100
Enter ratings limit for getting the subset of the dataset 
 (To get faster output please give a number between 100 and 500) 100


In [3]:
class matrix_factorization():
    def __init__(self,RInputMatrix,userFeatures,itemFeatures,K,iterations,alpha):
        # RInputMatrix : Input Data Matrix
        self.RInputMatrix = RInputMatrix
        # P: userFeatures(no: of users) * K(no: of latent features) = (user-features matrix)
        self.userFeatures = userFeatures
        # itemFeatures: I(no: of Items) * K(no: of latent features) = (item=features matrix)
        #Transposing the itemFeatures matrix to match with the order of User Features for matrix multiplication
        self.itemFeatures =itemFeatures.T
        # K: latent features
        self.K = K
        # iterations : default value 50000
        self.iterations = iterations
        # alpha: default value 0.0002
        self.alpha = alpha
        # beta: default value 0.02
        self.beta = float(0.02)
        
    def gradient_descent(self):
        #iterating over number of iterations
        for iteration in range(self.iterations):
            #iterating over number of rows in the given input matrix
            for i in range(self.RInputMatrix.shape[0]):
                #iterating over number of columns in the given input matrix
                for j in range(self.RInputMatrix.shape[1]):
                    #entering the if case if the value of RInputMatrix[rowNo][colNo] is greater than zero
                    if self.RInputMatrix[i][j] > 0 :
                        # calculating error i.e, (orginal value at m,n minus product of userFeatures and itemFeatures at m,n)
                        eij = self.RInputMatrix[i][j] - np.dot(self.userFeatures[i,:],self.itemFeatures[:,j])
                        sum_of_norms = 0
                        sum_of_norms += LA.norm(self.userFeatures) + LA.norm(self.itemFeatures)
                        eij += ((self.beta/2) * sum_of_norms)
                        for k in range(self.K):
                            # calculating gradient with alpha and beta parameter using the given formula
                            self.userFeatures[i][k] = self.userFeatures[i][k] + self.alpha * ( 2 * eij * self.itemFeatures[k][j] - (self.beta * self.userFeatures[i][k]))
                            self.itemFeatures[k][j] = self.itemFeatures[k][j] + self.alpha * (2 * eij * self.userFeatures[i][k] - (self.beta * self.itemFeatures[k][j]))
            #intialzing the errorVal to zero
            error = 0
            #iterating through number of rows in RInputMatrix
            for i in range(self.RInputMatrix.shape[0]):
                #iterating through number of columns in RInputMatrix
                for j in range(self.RInputMatrix.shape[1]):
                    #if RInputMatrix[m][n] is greater than zero  
                    #then calculating the error according to the updated userFeatures and itemFeatures
                    if self.RInputMatrix[i][j] > 0:
                        error += np.power(self.RInputMatrix[i][j] - np.dot(self.userFeatures[i,:],self.itemFeatures[:,j]),2)
            #Breaking the for loop after reaching the minimum error
            if error < 0.001:
                break
        #returing the userFeature and itemFeature matrices
        return self.userFeatures, self.itemFeatures.T

In [4]:
def matrixFacResult(X,K,iterations):
    # NRows: number of Users/rows in given matrix data
    NRows= X.shape[0]
    # MColumns: num of columns in given matrix data
    MColumns = X.shape[1]
    #K = Num of Latent Features
    #randomly constructing the userFeature matrix according to number of users(rows) and k-latent features(columns)
    userFeatures = np.random.rand(NRows,K)
    #randomly constructing the itemFeatures matrix according to number of columns of input matrix and k-latent features(columns)
    itemFeatures = np.random.rand(MColumns,K)
    steps = iterations
    alpha = 0.0002
    
    #calculating the minimum error bases userFeatures and ItemFeatures matrixes
    mf = matrix_factorization(X,userFeatures,itemFeatures,K,steps,alpha)
    userFeatures, itemFeatures = mf.gradient_descent()
    #calculating the output matrix by doing dot product of userFeatures matrix and transpose of ItemFeatures matrix
    recommendedRatings = np.dot(userFeatures,itemFeatures.T)
    return recommendedRatings

In [5]:
#Reading the ratings.csv file and saving it to a dataframe
ratingDataframe =pd.read_csv("ml-20m/ratings.csv")
#Getting total records in ratings
TotalRecords = len(ratingDataframe)
#filtering the ratings dataframe with max userId and movieId
ratingDataframe=ratingDataframe[(ratingDataframe["userId"]<=RatingsLimt) & (ratingDataframe["movieId"]<=moviesLimt)]
#getting the totals records after filter is applied on dataframe
ratingShape = len(ratingDataframe)

In [6]:
#Reading the movies.csv file and saving it to a dataframe
movieDataframe=pd.read_csv("ml-20m/movies.csv")
#filtering the movies dataframe with max userId provided by user
movieDataframe=movieDataframe[movieDataframe["movieId"]<=moviesLimt]
#getting the total records of filtered movie datafra
moviesShape = len(movieDataframe)

In [7]:
#Joining the ratingDataframe and movieDataframe on movieId
joinMovieRating = pd.merge(ratingDataframe, movieDataframe, on='movieId')
columns = ['genres','timestamp']
#Removing the coulmns genres, timestamp as they are not necessary
joinMovieRating = joinMovieRating.drop(columns, axis=1)
#Filling all the values with 
joinMovieRatingPivot = joinMovieRating.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)
joinMovieRatingPivot

title,Ace Ventura: When Nature Calls (1995),"American President, The (1995)",Angels and Insects (1995),Assassins (1995),Babe (1995),Beautiful Girls (1996),Bed of Roses (1996),Bio-Dome (1996),Black Sheep (1996),Broken Arrow (1996),...,To Die For (1995),Tom and Huck (1995),Toy Story (1995),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Two Bits (1995),Two if by Sea (1996),"Usual Suspects, The (1995)",Vampire in Brooklyn (1995),Waiting to Exhale (1995),White Squall (1996)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.5,4.0,0.0,0.0,3.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#Converting the pivot table to 2D numpy array as its needed to send to matrixFactorization as input
inputMatrix = joinMovieRatingPivot.values
inputMatrix = np.array(inputMatrix)

In [9]:
#Taking number of iterations from the user.
iterations = int(input("Enter Number of Iterations: (best if:250) "))
#Taking number of users want top10 movie recommendations 
userID = int(input("Enter User ID for who requested for movie Recommendations: (Enter Numbers only) "))

#Taking the K value from the user
K = int(input("Enter the K value: (best if:8) "))

#Taking the start time when the recommendation systems starts
startTime = time.time()

#getting the predicted matrix using matrixFacResult
resultMatrix = matrixFacResult(inputMatrix,K,iterations)        
    
#Taking the end time and displaying the total execution time
executionTime = (time.time() - startTime)

Enter Number of Iterations: (best if:250) 250
Enter User ID for who requested for movie Recommendations: (Enter Numbers only) 75
Enter the K value: (best if:8) 8


In [10]:
#Getting the ratings of the user requested 
userRatings = resultMatrix[int(userID)-1]

In [11]:
#Converting the userRatings to list
userRatings = userRatings.tolist()
#Getting the column names of input matrix
movieNames = joinMovieRatingPivot.columns.tolist()
#intialzing a dictonary to store movie name and its respective rating
movieRatings = dict()
count = 0
#iterating through each movieName and storing key as movie name and rating as value
for name in movieNames:
    movieRatings[name] = userRatings[count]
    count = count + 1
#Sorting the ratings in the dictonary in descending order
movieRatings = sorted(movieRatings.items(), key=lambda x: x[1], reverse=True)

In [12]:
count = 0
print("Top 10 movie recommendations for the requested user are:\n")
#iterating through the updated dictonary till we get the top 10 rated movies of the user
for key, value in movieRatings:
    #Showing the movie names to the user
    print(key)
    #breaking if 10 movie names are displayed
    if count == 9:
        break
    count = count + 1

print("\n\n")
    
#Showing the total execution time to the user
print('Execution time in seconds: ' + str(executionTime))

print("no:of ratings records:",ratingShape)
print("no: of movie records:",moviesShape)
print("Total Matrix elements",ratingShape*moviesShape)

Top 10 movie recommendations for the requested user are:

Usual Suspects, The (1995)
Postman, The (Postino, Il) (1994)
Seven (a.k.a. Se7en) (1995)
Toy Story (1995)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Dead Man Walking (1995)
American President, The (1995)
Heat (1995)
Bed of Roses (1996)
Get Shorty (1995)



Execution time in seconds: 13.017758131027222
no:of ratings records: 505
no: of movie records: 99
Total Matrix elements 49995


In [13]:
#initialzing a list to store the positions of intial rated movies per user
nonZeroRatingsUser = list()
#geting rows and colums
nonzeroInputMatrix = np.nonzero(inputMatrix)
#extracting rows
nonzeroInputMatrixRow = nonzeroInputMatrix[0]
#extracing columns
nonzeroInputMatrixCol = nonzeroInputMatrix[1]

#iterating and storing the nonZero
for row, col in zip(nonzeroInputMatrixRow, nonzeroInputMatrixCol):
    nonZeroRatingsUser.append((row, col))

In [14]:
#Intializing a list to store accuracies
accuracys = list()
#iterating through nonZeroRatingsUser dictonary
for key,value in nonZeroRatingsUser:
    try:
        
        #getting the user value from the input matrix
        userValue = inputMatrix[key][value]
        if userValue == 0:
          raise "skip"
        #getting predicted value from the predicted matrix
        predValue = resultMatrix[key][value]
        #Calculating the percentage change 
        if userValue > predValue:
            accuracys.append((userValue-predValue)/userValue)
        else:
            accuracys.append((predValue-userValue)/predValue)
    except:
        continue
        
#calculating the accuracy of the recommendation system
totalAccuracy = 0
for accuracy in accuracys:
    totalAccuracy = totalAccuracy + accuracy
print("Accuracy of recommendation system is:", (1-(totalAccuracy/len(accuracys)))*100)

Accuracy of recommendation system is: 84.93438625152211
